# 使用telomerehunter估计端粒长度
端粒是位于染色体末端的DNA-蛋白质复合体，在人出生时，其长度为8-15kb，每次复制之后，其长度会减少50-200bp。当端粒长度达到临界长度后，会导致复制性衰老和细胞衰亡。因此研究端粒长度对研究人类健康与疾病非常重要。

在这里，我们将为您呈现对2019年发布的计算端粒长度的算法[telomerehunter](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-019-2851-0)进行复现的过程，并对所得结果进行详细解释。

点击此处可以获得[telomerehunter](https://www.dkfz.de/en/applied-bioinformatics/telomerehunter/telomerehunter.html)的GitHub链接

# telomerehunter的背景

TelomereHunter是一个用于从人类全基因组测序数据中估算端粒含量的工具。**它被设计用于接收来自肿瘤和匹配对照样本的BAM文件作为输入。但也可以使用一个输入文件来运行TelomereHunter以计算端粒含量**。TelomereHunter从输入样本中提取并排序端粒reads，同时在估算端粒含量时，考虑了GC偏差。最后，TelomereHunter的结果以多个图表形式进行可视化呈现。

**在不考虑肿瘤组织的科研课题中，我们只需要输入单一Bam文件即可使用telomerehunter计算端粒长度。**




# telomerehunter的计算原理

没有经过GC校正(Bam输入)的端粒长度按照以下公式进行计算：

<math xmlns="http://www.w3.org/1998/Math/MathML">
  <mrow>
    <mi>Telomere_Length</mi>
    <mo>=</mo>
    <mfrac>
      <mi>Intratelomeric_Reads</mi>
      <mi>All_Reads</mi>
    </mfrac>
  </mrow>
</math>

经过GC校正的端粒长度按照以下公式进行计算：

<math xmlns="http://www.w3.org/1998/Math/MathML">
  <mrow>
    <mi>Telomere_Length</mi>
    <mo>=</mo>
    <mfrac>
      <mi>Intratelomeric_Reads</mi>
      <mi>Reads_with_GC_48-52%</mi>
    </mfrac>
  </mrow>
</math>


如下图所示，telomerehunter从BAM格式的NGS数据中提取包含较高端粒重复序列的reads。

提取得到包含较高端粒重复序列的reads后，telomerehunter使用来自BAM文件的比对信息对在第一步中选定的reads进行进一步的细分，将它们分为四个不同的类别。这些类别描述了reads在基因组中的位置和特性：

*  **内端粒 (Intratelomeric)**：这一类别包括那些在端粒区域内部的reads。端粒是染色体末端的特殊序列，因此在这个类别中的reads表示它们包含了端粒序列，并定位在染色体末端。（也是公式中的Intratelomeric_Reads）

*  **跨接口 (Junction Spanning)**：这一类别包括那些横跨端粒区域和其他区域的reads。这些reads可能横跨了端粒与非端粒区域之间的界限。

*  **亚端粒 (Subtelomeric)**：亚端粒reads是在端粒区域附近，但并不在端粒内部的reads。它们位于染色体末端的接近端粒的区域，但不同于真正的端粒序列。

*  **染色体内 (Intrachromosomal)**：这一类别包括那些在染色体内部的reads，即不在端粒区域也不跨越端粒区域的reads。这些reads定位在染色体的非末端区域。

![Image title](./img/pipeline.png)


# 下载，安装和运行telomerehunter以计算端粒长度

在此，我们将介绍如何下载telomerehunter，如何使用miniconda创建telomerehunter能够运行的环境，如何调试环境以让telomerehunter正确运行。

**请务必注意，telomerehunter是基于python2.0开发的**，我们创建环境的时候不可以创建python3.0以及以上版本的环境，否则无法安装telomerehunter。

以下是作者给出的环境依赖需求：

 for telomere read extraction and calculation of telomere content
  * python 2.7.9 (does not work for python 3!)
    * pysam 0.9.0
    * PyPDF2 1.26.0
  * samtools 1.3.1

In [ ]:
# 创建新环境
conda create -n telomerehunter python=2.7
# 启动环境
conda  activate telomerehunter

由于作者并没有给出telomerehunter的Github链接，所以我们无法利用传统的git clone方法安装telomerehunter。

但是作者给出了上传在pypi的telomerehunter.whl文件的下载链接。我们可以手动下载.whl文件来安装telomerehunter，我们可以点击此处下载[telomerehunter](https://pypi.org/project/telomerehunter/#files)。


**.whl 文件（Wheel 文件）是Python软件包的分发格式之一，用于简化Python包的安装和分发。**Wheel 文件的目标是提高Python包的安装效率和可移植性，以替代传统的.tar.gz或.zip源代码分发包。

* 具体来说，.whl 文件是二进制分发包，其中包含了已编译的Python包，这意味着它们通常更容易安装，因为不需要编译源代码。

* 对于.whl文件来讲，Python的包管理工具pip，可以直接安装.whl 文件。这使得通过pip安装Python包变得更加简单。

所以我们使用以下代码，下载和安装telomerehunter


In [ ]:
# 我们首先创建一个文件夹，用来存放telomerehunter.whl文件。
mkdir telomerehunter
cd telomerehunter

# 使用wget指令下载TelomereHunter的安装包
wget https://files.pythonhosted.org/packages/e5/67/ce6ac292a88a078a733dc3d9adb3f153834692effbf0851b93a6f3e49b7a/telomerehunter-1.1.0-py2-none-any.whl

# 使用pip指令安装TelomereHunter
pip install telomerehunter-1.1.0-py2-none-any.whl

此外运行telomerehunter还需要samtools。

西湖大学高性能计算中心提供了samtools的依赖支持，我们输入以下代码只需要手动启动samtools即可。

In [ ]:
module load samtools/1.16.1  

# 运行telomerehunter以估计样品的平均端粒长度

你可以使用命令行或西湖大学高性能计算平台的srun系统来运行telomerehunter。

下面提供了一个在srun系统中运行telomerehunter的模拟代码示例。在这个示例中，我们将计算特定路径下所有的BAM文件的平均端粒长度，并将结果输出到我们手动指定的目标文件夹中。

请注意，我们只需要指定包含输入bam文件的文件夹，和保存输出结果的目标文件夹。

如果你想查找**telomerehunter工具包的其他参数及其默认参数**，你可以在**命令行**中输入`telomerehunter -h`，即可得到相关参数说明。

请注意，由于telomerehunter每次计算只能计算一个文件，且输出一个单独的文件夹，**意味着如果你输入多个文件，telomerehunter的结果会保存在多个文件夹的****_summary.tsv文件中**，如下图所示，所以我们需要对srun系统中的运行代码进行修改，手动指定输出的文件路径。


![Image title](./img/telomerehunter_srun.png)

In [ ]:
#!/bin/bash
#SBATCH -p intel-sc3,amd-ep2
#SBATCH -q normal
#SBATCH -J preprocess			
#SBATCH -c 4		
#SBATCH -o /home/douyanmeiLab/hulei/WGS/telomere/Final_output/log/telomerehunter.log
#SBATCH --mem=20G
#SBATCH -t 120:00:00

# 设置样本文件夹和输出文件夹的路径
conda activate telomerehunter
module load samtools/1.16.1  


input_folder="/storage/douyanmeiLab/hulei/Data/Bam"
output_folder="/home/douyanmeiLab/hulei/WGS/telomere/Final_output/telomerehunter"

# 列出样本文件夹中的所有.bam文件
bam_files=("$input_folder"/*.bam)

# 循环处理每个.bam文件
for bam_file in "${bam_files[@]}"; do
    # 提取.bam文件的文件名（不包括扩展名）作为样本名称
    sample_name=$(basename "$bam_file" .bam)

    # 打印样本名称
    echo "Processing sample: $sample_name"

    # 运行telomerehunter命令
    telomerehunter -ibc "$bam_file" -p "$sample_name" -o "$output_folder"
done


# telomerehunter结果说明

telomerehunter的输出结果是一个****_summary.tsv文件。每个文件包含一行的columns信息和一行的输出结果信息。

对于这种类型的文件，你可以在Linux中手动打开它，或者创建一个IPython Notebook文件，并使用Pandas库来打开CSV文件。****_summary.tsv文件的内容如下：

![Image title](./img/result.png)


在此，我们简单介绍一下telseq输出文件中比较重要的几个结果。

* PID：该参数由BAM头文件中的RG tag信息所确定，一般为样本的ID信息。

* intratel_reads：被识别来自于内端粒的Reads数量。

* total_reads_with_tel_gc：被识别gc含量在48-52%的reads数量。

* **tel_content：telomerehunter估计的端粒长度，实际上这个长度是由intratel_reads*10^6/total_reads_with_tel_gc计算而来的。**

我们可以写一个python代码，手动读取目标文件夹下面所有telomerehunter的输出的****_summary.tsv文件，然后将每个文件的输出结果合并在同一个tsv文件中，相关代码如下：

在这个代码中，我们只需要指定保存telomerehunter的输出结果的文件夹和保存合并后的tsv文件的路径即可。这个代码会手动合并所有结果。

In [ ]:
import os
import pandas as pd

# 目标文件夹路径
target_folder = "/home/douyanmeiLab/hulei/WGS/telomere/Final_output/telomerehunter"
output_file = "/home/douyanmeiLab/hulei/WGS/telomere/Final_output/telomerehunter/merged_summary.tsv"

# 初始化一个空的DataFrame来存储合并的数据
merged_data = pd.DataFrame()

# 遍历目标文件夹及其子文件夹
for root, dirs, files in os.walk(target_folder):
    for file in files:
        if file.endswith("_summary.tsv"):
            file_path = os.path.join(root, file)
            # 读取每个子文件夹中的_summary.tsv文件内容
            data = pd.read_csv(file_path, delimiter='\t')
            merged_data = pd.concat([merged_data, data], axis=0)

# 保存合并后的数据到一个新的tsv文件
merged_data.to_csv(output_file, sep='\t', index=False)

print(f"合并完成，合并后的数据保存在{output_file}中。")


代码的输出结果如下图所示：

![Image title](./img/pandas.png)

基于这个输出结果，我们可以利用merged_summary.csv文件中的tel_content的数值来绘制估计的端粒长度与年龄之间的非线性相关关系。如下图所示.

![Image title](./img/telomerehunter_plot.png)

以上便是本教程的全部内容了，希望你能在此教程中有所收获，能够学会如何正确下载，安装，编译和使用telomerehunter软件，并且估计输入Bam文件对应的端粒长度。